In [1]:
import pandas as pd
import json
import re

In [2]:
# Configuration

league = 'Mercenaries'
league_type = 'exp'
min_listing_count = 100
min_ilvl = 80
skills = ['Earthshatter', 'Cyclone', 'Smite of Divine Judgement', 'Righteous Fire', 'Forbidden Right of Soul Sacrifice', 'Kinetic Blast', 'Flicker Strike', 'Blight of Contagion']
character_examples_per_skill = 50

number_pattern = r'\d+(\.\d+)?'

In [3]:
# Get prices for cluster gem bases

uri = "https://poe.ninja/api/data/itemoverview?league=" + league + "&type=ClusterJewel"

df = pd.read_json(uri)
df = df['lines'].apply(pd.Series)
df = df[(df['listingCount'] > min_listing_count)]
# It's not fully accurate to exclude ilvl from consideration, but it makes us duplicate rows later if we leave it in.
df = df[(df['levelRequired'] > min_ilvl)]
df = df[['name', 'baseType', 'variant', 'chaosValue', 'divineValue', 'listingCount']]

df['name'] = df['name'].str.replace(number_pattern, 'X', regex=True)

df['variant'] =  df['variant'].str.extract(r'(\d+)')
df = df.rename(columns={'name': 'base', 'baseType': 'type', 'variant': 'passives'})

df = df.sort_values(by='chaosValue', ascending=False).reset_index(drop=True)
df

,base,type,passives,chaosValue,divineValue,listingCount
0,X% increased Attack Damage while holding a Shield,Large Cluster Jewel,12,101.20,0.48,7557
1,Minions deal X% increased Damage,Large Cluster Jewel,12,100.00,0.48,21185
2,X% increased Lightning Damage,Large Cluster Jewel,12,75.00,0.36,12474
3,X% increased Attack Damage while holding a Shield,Large Cluster Jewel,8,60.00,0.29,13044
4,X% increased Spell Damage,Large Cluster Jewel,12,50.57,0.24,20354
...,...,...,...,...,...,...
165,X% Chance to Block Spell Damage,Small Cluster Jewel,3,10.00,0.05,6551
166,+X% Chance to Block Attack Damage,Small Cluster Jewel,2,10.00,0.05,5036
167,X% Chance to Block Spell Damage,Small Cluster Jewel,2,10.00,0.05,4413
168,+X% chance to Suppress Spell Damage,Small Cluster Jewel,3,10.00,0.05,14939


In [4]:
# Fetch the list of links to top characters by DPS

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# Set up the service and launch the browser
options = Options()
options.add_argument('--headless')
service = Service(ChromeDriverManager().install())

# Get character links for each primary skill
character_links = []
skill_url_substrings = [skill.replace(' ', '+') for skill in skills]

driver = webdriver.Chrome(service=service, options=options)
try:
    for skill in skill_url_substrings:
        build_uri = 'https://poe.ninja/builds/' + league.lower() + '?skills=' + skill + '&sort=dps'
        driver.get(build_uri)
        
        driver.implicitly_wait(1)

        # Clean it up
        links = driver.find_elements(By.XPATH, "//a[contains(@href, '/builds/mercenaries/character/')]")
        links = [link.get_attribute('href').partition('?i=')[0] for link in links]
        
        character_links.extend(links[:character_examples_per_skill])
finally:
    driver.quit()

# Important to drop NaN or the fetch will blow up later
char_url_parts = pd.DataFrame([link.split('/')[-2:] for link in character_links], columns=['account', 'character']).dropna().reset_index(drop=True)
char_url_parts

,account,character
0,xXArck3nXx-7670,GrotSlaveLabourer
1,PureAmericanSalt-0778,MercenariesMelvin
2,Arvandorpros-4526,arvmercslam
3,ReveN_RV-2143,WLebSieTaMaczugaJebnij
4,Hajopu-3008,InffiSlams
...,...,...
395,cristian23-6024,Zetna
396,livejamie-0512,livejamie_wearesoback
397,Kantorn13-5889,Icetoh
398,Dutay-2799,ChaosWitchDubtay


In [5]:
# Get the most recent snapshot version for profile requests

import requests

snapshots = json.loads(requests.get('https://poe.ninja/poe1/api/data/index-state').text)
snapshots = pd.json_normalize(snapshots['snapshotVersions']).apply(pd.Series)
snapshot_version = snapshots[(snapshots['name'] == league) & (snapshots['type'] == league_type.lower())]['version'][0]
snapshot_version

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
# Parse the cluster jewels from a player's public profile

def fetch_cluster_jewels(account, character):
    char_uri = 'https://poe.ninja/api/data/' + snapshot_version + '/getcharacter?account=' + account + '&name=' + character + '&overview=' + league.lower() + '&type=' + league_type.lower()
    whitelisted_columns = ['itemData.baseType', 'itemData.enchantMods', 'itemData.explicitMods']

    try:
        char = json.loads(requests.get(char_uri).text)
        
        jewels = pd.json_normalize(char['jewels'])
        jewels = jewels[jewels['itemData.baseType'].str.contains('Cluster')].reset_index(drop=True)
        jewels = jewels[whitelisted_columns]
        jewels.columns = ['type', 'base', 'mods']
        jewels['passives'] = jewels['base'].str[0].str.extract(r'(\d+)')
        jewels = jewels[['type', 'passives', 'base', 'mods']]
        
        jewels['base'] = jewels['base'].str[2].str.replace('Added Small Passive Skills grant: ', '', regex=False)
        jewels['base'] = jewels['base'].str.replace(number_pattern, 'X', regex=True)

        newline_pattern = r'\n'
        jewels['base'] = jewels['base'].str.replace(newline_pattern, ', ', regex=True)
        
        jewels['mods'] = jewels['mods'].apply(lambda mod_list: [re.sub(number_pattern, 'X', mod.removeprefix('Added Small Passive Skills also grant: ')) for mod in mod_list])
    finally:
        # We don't care about uniques like Voices, which will create parse failures in the base and passives columns.
        return jewels.dropna()

jewels = pd.concat(char_url_parts.apply(lambda row: fetch_cluster_jewels(row['account'], row['character']), axis=1).tolist(), ignore_index=True)
jewels

,type,passives,base,mods
0,Large Cluster Jewel,8,Staff Attacks deal X% increased Damage with Hi...,"[Regenerate X% of Life per Second, X Added Pas..."
1,Medium Cluster Jewel,4,Exerted Attacks deal X% increased Damage,"[Regenerate X% of Life per Second, X Added Pas..."
2,Medium Cluster Jewel,4,Exerted Attacks deal X% increased Damage,"[+X to All Attributes, X Added Passive Skill i..."
3,Large Cluster Jewel,8,X% increased Physical Damage,"[Regenerate X% of Life per Second, X Added Pas..."
4,Large Cluster Jewel,8,X% increased Physical Damage,"[+X to Intelligence, X Added Passive Skill is ..."
...,...,...,...,...
1168,Medium Cluster Jewel,4,X% increased Chaos Damage over Time,"[+X to All Attributes, X% increased Mana Regen..."
1169,Large Cluster Jewel,8,X% increased Chaos Damage,"[+X% to all Elemental Resistances, X Added Pas..."
1170,Medium Cluster Jewel,4,X% increased Chaos Damage over Time,"[+X% to Chaos Resistance, +X% to Fire Resistan..."
1171,Medium Cluster Jewel,4,X% increased Chaos Damage over Time,"[+X to Dexterity, X Added Passive Skill is Vil..."


In [7]:
exploded = jewels.explode('mods')

# I have no idea where there are lists here, but they cause the count to fail, so drop them I guess?
mask = exploded.apply(lambda x: any(isinstance(val, list) for val in x), axis=1)
exploded = exploded[~mask]

mod_counts = exploded.groupby(['type', 'base', 'passives'])['mods'].value_counts().reset_index()
mod_counts = mod_counts.sort_values(by='count', ascending=False)
mod_counts

,type,base,passives,mods,count
106,Large Cluster Jewel,X% increased Attack Damage while holding a Shield,8,X Added Passive Skill is Martial Prowess,106
215,Large Cluster Jewel,X% increased Fire Damage,12,+X to All Attributes,104
216,Large Cluster Jewel,X% increased Fire Damage,12,+X to Strength,99
217,Large Cluster Jewel,X% increased Fire Damage,12,+X to Maximum Life,97
244,Large Cluster Jewel,X% increased Lightning Damage,12,X% increased Attack and Cast Speed with Lightn...,96
...,...,...,...,...,...
537,Medium Cluster Jewel,X% increased Projectile Damage,5,X% increased Mana Regeneration Rate,1
538,Medium Cluster Jewel,X% increased Totem Damage,5,+X to Dexterity,1
539,Medium Cluster Jewel,X% increased Totem Damage,5,X Added Passive Skill is Ancestral Echo,1
540,Medium Cluster Jewel,X% increased Totem Damage,5,X Added Passive Skill is Sleepless Sentries,1


In [8]:
final = pd.merge(mod_counts, df, on=['base', 'type', 'passives']).sort_values(by='count', ascending=False)
final = final.rename(columns={'chaosValue': 'base chaos', 'divineValue': 'base div', 'listingCount': 'base count'})
final

,type,base,passives,mods,count,base chaos,base div,base count
0,Large Cluster Jewel,X% increased Attack Damage while holding a Shield,8,X Added Passive Skill is Martial Prowess,106,60.0,0.29,13084
1,Large Cluster Jewel,X% increased Fire Damage,12,+X to All Attributes,104,30.0,0.14,11827
2,Large Cluster Jewel,X% increased Fire Damage,12,+X to Strength,99,30.0,0.14,11827
3,Large Cluster Jewel,X% increased Fire Damage,12,+X to Maximum Life,97,30.0,0.14,11827
5,Large Cluster Jewel,X% increased Attack Damage while holding a Shield,8,X Added Passive Skill is Prodigious Defence,96,60.0,0.29,13084
...,...,...,...,...,...,...,...,...
523,Medium Cluster Jewel,X% increased Critical Strike Chance,5,+X% to Critical Strike Multiplier,1,19.2,0.09,13999
522,Medium Cluster Jewel,X% increased Critical Strike Chance,5,+X to Strength,1,19.2,0.09,13999
521,Medium Cluster Jewel,X% increased Critical Strike Chance,5,+X to Intelligence,1,19.2,0.09,13999
520,Large Cluster Jewel,Minions deal X% increased Damage,12,+X% to Chaos Resistance,1,100.0,0.48,21281


In [10]:
final.to_excel("~/git/PoE1ClusterJewelPricer/ClusterJewels.xlsx", sheet_name='Jewels', index=False)